Run this notebook to

- f


## Import stuff

In [ ]:
import os
import numpy as np
import mne
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs,corrmap)
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd
from scipy import stats, signal

import matplotlib as mpl

#%matplotlib qt
%matplotlib inline

sns.set_style('whitegrid')
mpl.rcParams['font.size'] = 16
mpl.rcParams['axes.titlesize'] = 'small'
mpl.rcParams['axes.labelsize'] = 'small'
mpl.rcParams['xtick.labelsize'] = 'small'
mpl.rcParams['ytick.labelsize'] = 'small'
mpl.rcParams['legend.fontsize'] = 'small'
mpl.rcParams['figure.titlesize'] = 'large'
mpl.rcParams['figure.figsize'] = (8, 5) # does not seem to work
mpl.rc('font', family='DejaVu Sans')
mpl.rcParams['svg.fonttype'] = 'none'

million = 1000000.



input_dir = 'TaskstimulusEpochsMastoids'


In [ ]:
participant_numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21]

sessions = [1, 2]

conditions = ["s", "ns"]

partners = ["overconfident", "underconfident"]

In [ ]:
stimulus_epochs = mne.read_epochs('mergedData/stimulus_epoch_mastoids-epo.fif')

In [ ]:
stimulus_epochs.metadata.index = range(len(stimulus_epochs.metadata))

In [ ]:
stimulus_epochs_Pz = stimulus_epochs.copy().pick('Pz')

In [ ]:
zscore = lambda x: (x - x.mean()) / x.std()
# zscore confidence by participant
stimulus_epochs.metadata['confidence_z_by_participant'] = stimulus_epochs.metadata['participant_confidence'].groupby(stimulus_epochs.metadata['participant']).transform(zscore)

## Calculate max amplitude and latency post stimulus at Pz

### using mne get_peak function

In [ ]:
for ix, trial in enumerate(stimulus_epochs_Pz.iter_evoked()):
    channel, latency, value = trial.get_peak(return_amplitude=True, mode='pos', tmin=0.25, tmax=1)
    latency = float(round(latency * 1e3, 5))  # convert to milliseconds
    value = float(round(value * 1e6, 5))      # convert to µV
    print('Trial {}: max of {} µV at {} ms in channel {}'
          .format(ix, value, latency, channel))
    stimulus_epochs.metadata.loc[ix, 'Pz_max_voltage'] = value
    stimulus_epochs.metadata.loc[ix, 'Pz_max_latency'] = latency

In [ ]:
plt.title("max voltage", fontsize=20)
plt.xlabel("max voltage at Pz 250-1000ms post stimulus")
plt.ylabel("count")
bins = np.arange(stimulus_epochs.metadata['Pz_max_voltage'].min(), stimulus_epochs.metadata['Pz_max_voltage'].max()+1, 1)
plt.xlim((-10, 40))
plt.hist(stimulus_epochs.metadata['Pz_max_voltage'], bins=bins)
plt.show() 

In [ ]:
plt.title("latency", fontsize=20)
plt.xlabel("latency of max voltage at Pz 250-1000ms post stimulus")
plt.ylabel("count")
bins = np.arange(stimulus_epochs.metadata['Pz_max_latency'].min(), stimulus_epochs.metadata['Pz_max_latency'].max()+4, 4)
plt.hist(stimulus_epochs.metadata['Pz_max_latency'], bins=bins)
plt.show() 

### using numpy amax and argmax (gives the same results as the mne get_peak function)

In [ ]:
tmax = 1000
tmin = 250

indexmax = round(int(300+tmax/4)) # index 0 is at -300ms; to convert index to ms, conider that this is at 250Hz, i.e. 4ms increase means 1 index increase
indexmin= round(int((300+tmin)/4))
i = 0

for epoch in stimulus_epochs_Pz:
    epoch = epoch.flatten()
    epoch = epoch[indexmin:indexmax] 
    max = round(np.amax(epoch) * 1e6, 5)
    latency = round(np.argmax(epoch)*4+tmin, 5)
    stimulus_epochs.metadata.loc[i, 'Pz_max_voltage_manual'] = max
    stimulus_epochs.metadata.loc[i, 'Pz_max_latency_manual'] = latency
    channel = "Pz"
    print('Trial {}: max of {} µV at {} ms in channel {}'
          .format(i, max, latency, channel))
    i +=1

In [ ]:
plt.title("max voltage", fontsize=18)
plt.xlabel("max voltage at Pz 250-1000ms post stimulus")
plt.ylabel("count")
bins = np.arange(stimulus_epochs.metadata['Pz_max_voltage_manual'].min(), stimulus_epochs.metadata['Pz_max_voltage_manual'].max()+1, 1)
plt.xlim((-10, 40))
plt.hist(stimulus_epochs.metadata['Pz_max_voltage_manual'], bins=bins)
plt.show() 

In [ ]:
plt.title("latency", fontsize=18)
plt.xlabel("latency of max voltage at Pz 250-1000ms post stimulus")
plt.ylabel("count")
bins = np.arange(stimulus_epochs.metadata['Pz_max_latency_manual'].min(), stimulus_epochs.metadata['Pz_max_latency_manual'].max()+4, 4)
plt.hist(stimulus_epochs.metadata['Pz_max_latency_manual'], bins=bins)
plt.show() 

## Throw out edge artifact trials

In [ ]:
print("Original number of trials: %i" % len(stimulus_epochs))

In [ ]:
stimulus_epochs_clear_peak = stimulus_epochs['Pz_max_latency > %i' % 254]
a = len(stimulus_epochs_clear_peak)
print("Number of trials excluded for just drifting down: %i" % (len(stimulus_epochs) - a))

In [ ]:
stimulus_epochs_clear_peak = stimulus_epochs_clear_peak['Pz_max_latency < %i' % 996]
print("Number of trials excluded for just drifting up: %i" % (a - len(stimulus_epochs_clear_peak)))

In [ ]:
plt.title("max voltage", fontsize=18)
plt.xlabel("max voltage at Pz 250-1000ms post stimulus")
plt.ylabel("count")
bins = np.arange(stimulus_epochs_clear_peak.metadata['Pz_max_voltage'].min(), stimulus_epochs_clear_peak.metadata['Pz_max_voltage'].max()+1, 1)
plt.xlim((-10, 40))
plt.hist(stimulus_epochs_clear_peak.metadata['Pz_max_voltage'], bins=bins)
plt.show() 

In [ ]:
print("Mean max voltage: %.2f \n\nMedian max voltage: %.2f \n\nMax max voltage: %.2f \n\nMin max voltage: %.2f \n" 
      % (np.mean(stimulus_epochs_clear_peak.metadata['Pz_max_voltage']), np.median(stimulus_epochs_clear_peak.metadata['Pz_max_voltage']),
        np.max(stimulus_epochs_clear_peak.metadata['Pz_max_voltage']), np.min(stimulus_epochs_clear_peak.metadata['Pz_max_voltage'])))

In [ ]:
plt.title("latency", fontsize=18)
plt.xlabel("latency of max voltage at Pz 250-1000ms post stimulus")
plt.ylabel("count")
bins = np.arange(stimulus_epochs_clear_peak.metadata['Pz_max_latency'].min(), stimulus_epochs_clear_peak.metadata['Pz_max_latency'].max()+4, 4)
plt.hist(stimulus_epochs_clear_peak.metadata['Pz_max_latency'], bins=bins)
plt.show() 

In [ ]:
print("Mean max voltage latency: %.2f ms post stimulus\n \nMedian max voltage latency: %.2f ms post stimulus \n \nMax max voltage latency: %.2f ms post stimulus \n \nMin max voltage latency: %.2f ms post stimulus \n" % 
      (np.mean(stimulus_epochs_clear_peak.metadata['Pz_max_latency']), np.median(stimulus_epochs_clear_peak.metadata['Pz_max_latency']),
      np.max(stimulus_epochs_clear_peak.metadata['Pz_max_latency']), np.min(stimulus_epochs_clear_peak.metadata['Pz_max_latency'])))

In [ ]:
print("\n----------------------- all participants -----------------------\n\n")
for condition in conditions:
    condition_epochs = stimulus_epochs_clear_peak['condition == "%s"' %  condition]
    if condition == "ns":
        print("observational condition")
    else:
        print("strategic condition")
    
    for partner in partners:
        partner_epochs = condition_epochs['partner == "%s"' %  partner]
        x=partner_epochs.metadata['confidence_z_by_participant']
        y=partner_epochs.metadata['Pz_max_voltage']
        slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
        line = slope*x+intercept
        #plt.plot(x, line, label='{} y={:.2f}x+{:.2f}'.format(partner,slope,intercept))
        plt.plot(x, line, label='%s' % partner)
        plt.xlim(-4, 4)
        plt.scatter(x,y, color="k", s=2, alpha=0.2)
        plt.legend(fontsize=13)
        plt.xlabel("reported confidence z-scored by participant")
        plt.ylabel("max voltage at Pz")
    
    plt.show()

In [ ]:
for participant in participant_numbers:
    participant_epochs = stimulus_epochs_clear_peak['participant == %i' %  participant]
    print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % participant)
    print('\033[0m')
    
    for condition in conditions:
        condition_epochs = participant_epochs['condition == "%s"' %  condition]
        if condition == "ns":
            print("observational condition")
        else:
            print("strategic condition")

        for partner in partners:
            partner_epochs = condition_epochs['partner == "%s"' %  partner]
            
            x=partner_epochs.metadata['participant_confidence']
            y=partner_epochs.metadata['Pz_max_voltage']
            
            slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
            line = slope*x+intercept
            
            #plt.plot(x, line, label='y={:.2f}x+{:.2f}'.format(slope,intercept))
            plt.plot(x, line, label='%s' % partner)
            plt.xlim(50, 100)
            plt.scatter(x,y, color="k", s=2, alpha=0.5)
            plt.legend(fontsize=13)
            plt.xlabel("reported confidence")
            plt.ylabel("max voltage at Pz")
            
        plt.show()

In [ ]:
correlations_list = []

for participant in participant_numbers:
    participant_epochs = stimulus_epochs_clear_peak['participant == %i' %  participant]
    
    for condition in conditions:
        condition_epochs = participant_epochs['condition == "%s"' %  condition]

        for partner in partners:
            partner_epochs = condition_epochs['partner == "%s"' %  partner]
            
            corr, _ = stats.pearsonr(partner_epochs.metadata['Pz_max_voltage'], partner_epochs.metadata['participant_confidence'])
            correlations_list.append(corr)
            
            
print('Average correlation between max voltage at Pz and reported confidence: %.4f' % np.mean(correlations_list))            

In [ ]:
correlations_list = []
correct_voltage_list = []
incorrect_voltage_list = []

for participant in participant_numbers:
    participant_epochs = stimulus_epochs_clear_peak['participant == %i' %  participant]
    correct_epochs = participant_epochs['participant_correct == True']
    correct_voltage_list.append(np.mean(correct_epochs.metadata['Pz_max_voltage']))
    incorrect_epochs = participant_epochs['participant_correct == False']
    incorrect_voltage_list.append(np.mean(incorrect_epochs.metadata['Pz_max_voltage']))
    
    for condition in conditions:
        condition_epochs = participant_epochs['condition == "%s"' %  condition]

        for partner in partners:
            partner_epochs = condition_epochs['partner == "%s"' %  partner]
            
            corr, _ = stats.pointbiserialr(partner_epochs.metadata['Pz_max_voltage'], partner_epochs.metadata['participant_correct'])
            correlations_list.append(corr)
            
            
print('\nAverage point biserial correlation between max voltage at Pz and accuracy: %.4f' % np.mean(correlations_list))  
print('\nAverage max voltage on correct trials: %.4f' % np.mean(correct_voltage_list))
print('\nAverage max voltage on inorrect trials: %.4f' % np.mean(incorrect_voltage_list))

In [ ]:
correct_epochs = stimulus_epochs_clear_peak['participant_correct == True']
incorrect_epochs = stimulus_epochs_clear_peak['participant_correct == False']
x1 = correct_epochs.metadata['Pz_max_voltage']
x2 = incorrect_epochs.metadata['Pz_max_voltage']

plt.title("max voltage", fontsize=18)
plt.xlabel("max voltage at Pz 250-1000ms post stimulus")
plt.ylabel("count")
bins = np.arange(stimulus_epochs.metadata['Pz_max_voltage'].min(), stimulus_epochs.metadata['Pz_max_voltage'].max()+1, 1)
plt.xlim((-10, 40))
plt.hist(x1, bins=bins, alpha=0.5, color='green', label='correct trials')
plt.axvline(x1.mean(), color='green', linestyle='dashed', linewidth=2)
plt.text(s='%.2f' % x1.mean(), y=520, x=x1.mean()+2, color='green')
plt.hist(x2, bins=bins, alpha=0.5, color='red', label='incorrect trials')
plt.axvline(x2.mean(), color='red', linestyle='dashed', linewidth=2)
plt.text(s='%.2f' % x2.mean(), y=520, x=x2.mean()-6, color='red')
plt.legend()
plt.show() 

## 

## Calculate highest peak amplitude and latency post stimulus at Pz

### i.e. instead of looking for the max voltage we look for peaks and take the highest one

In [ ]:
tmax = 1000
tmin = 250

indexmax = round(int((300+tmax)/4)) # index 0 is at -300ms; to convert index to ms, conider that this is at 250Hz, i.e. 4ms increase means 1 index increase
indexmin= round(int((300+tmin)/4))
i = 0

for epoch in stimulus_epochs_Pz:
    epoch = epoch.flatten()
    epoch = epoch[indexmin:indexmax] 
    indexes, properties = signal.find_peaks(epoch, height=[-100, 100])
    highest_peak = round(np.amax(properties['peak_heights']) * 1e6, 5) 
    latency = round(indexes[np.argmax(properties['peak_heights'])]*4+tmin, 5)
    number_of_peaks = len(indexes)
    stimulus_epochs.metadata.loc[i, 'Pz_highest_peak_voltage'] = highest_peak
    stimulus_epochs.metadata.loc[i, 'Pz_highest_peak_latency'] = latency
    stimulus_epochs.metadata.loc[i, 'Pz_number_of_peaks'] = number_of_peaks
    channel = "Pz"
    print('Trial {}: highest peak of {} µV at {} ms in channel {}'
          .format(i, highest_peak, latency, channel))
    i +=1



In [ ]:
plt.title("number of peaks", fontsize=18)
plt.xlabel("number of peaks at Pz 250-1000ms post stimulus")
plt.ylabel("count")
bins = np.arange(stimulus_epochs.metadata['Pz_number_of_peaks'].min(), stimulus_epochs.metadata['Pz_number_of_peaks'].max()+1, 1)
plt.hist(stimulus_epochs.metadata['Pz_number_of_peaks'], bins=bins)
plt.show() 

In [ ]:
plt.title("highest peak voltage", fontsize=18)
plt.xlabel("highest peak voltage at Pz 250-1000ms post stimulus")
plt.ylabel("count")
bins = np.arange(stimulus_epochs.metadata['Pz_highest_peak_voltage'].min(), stimulus_epochs.metadata['Pz_highest_peak_voltage'].max()+1, 1)
plt.xlim((-30, 60))
plt.hist(stimulus_epochs.metadata['Pz_highest_peak_voltage'], bins=bins)
plt.show() 

In [ ]:
stimulus_epochs.metadata['Pz_highest_peak_voltage']

In [ ]:
print("Mean highest peak voltage: %.2f \n\nMedian highest peak voltage: %.2f \n\nMax highest peak voltage: %.2f \n\nMin highest peak voltage: %.2f \n" 
      % (np.mean(stimulus_epochs.metadata['Pz_highest_peak_voltage']), np.median(stimulus_epochs.metadata['Pz_highest_peak_voltage']),
        np.max(stimulus_epochs.metadata['Pz_highest_peak_voltage']), np.min(stimulus_epochs.metadata['Pz_highest_peak_voltage'])))

In [ ]:
plt.title("latency", fontsize=18)
plt.xlabel("latency of highest peak at Pz 250-1000ms post stimulus")
plt.ylabel("count")
bins = np.arange(stimulus_epochs.metadata['Pz_highest_peak_latency'].min(), stimulus_epochs.metadata['Pz_highest_peak_latency'].max()+4, 4)
plt.hist(stimulus_epochs.metadata['Pz_highest_peak_latency'], bins=bins)
plt.show() 

In [ ]:
print("Mean highest peak latency: %.2f ms post stimulus\n \nMedian highest peak latency: %.2f ms post stimulus \n \nMax highest peak latency: %.2f ms post stimulus \n \nMin highest peak latency: %.2f ms post stimulus \n" % 
      (np.mean(stimulus_epochs.metadata['Pz_highest_peak_latency']), np.median(stimulus_epochs.metadata['Pz_highest_peak_latency']),
      np.max(stimulus_epochs.metadata['Pz_highest_peak_latency']), np.min(stimulus_epochs.metadata['Pz_highest_peak_latency'])))

In [ ]:
print("\n----------------------- all participants -----------------------\n\n")
for condition in conditions:
    condition_epochs = stimulus_epochs['condition == "%s"' %  condition]
    if condition == "ns":
        print("observational condition")
    else:
        print("strategic condition")
    
    for partner in partners:
        partner_epochs = condition_epochs['partner == "%s"' %  partner]
        x=partner_epochs.metadata['confidence_z_by_participant']
        y=partner_epochs.metadata['Pz_highest_peak_voltage']
        slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
        line = slope*x+intercept
        #plt.plot(x, line, label='{} y={:.2f}x+{:.2f}'.format(partner,slope,intercept))
        plt.plot(x, line, label='%s' % partner)
        plt.xlim(-4, 4)
        plt.scatter(x,y, color="k", s=2, alpha=0.2)
        plt.legend(fontsize=13)
        plt.xlabel("reported confidence z-scored by participant")
        plt.ylabel("highest peak voltage at Pz")
    
    plt.show()

In [ ]:
for participant in participant_numbers:
    participant_epochs = stimulus_epochs['participant == %i' %  participant]
    print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % participant)
    print('\033[0m')
    
    for condition in conditions:
        condition_epochs = participant_epochs['condition == "%s"' %  condition]
        if condition == "ns":
            print("observational condition")
        else:
            print("strategic condition")

        for partner in partners:
            partner_epochs = condition_epochs['partner == "%s"' %  partner]
            
            x=partner_epochs.metadata['participant_confidence']
            y=partner_epochs.metadata['Pz_highest_peak_voltage']
            
            slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
            line = slope*x+intercept
            
            #plt.plot(x, line, label='y={:.2f}x+{:.2f}'.format(slope,intercept))
            plt.plot(x, line, label='%s' % partner)
            plt.xlim(50, 100)
            plt.scatter(x,y, color="k", s=2, alpha=0.5)
            plt.legend(fontsize=13)
            plt.xlabel("reported confidence")
            plt.ylabel("highest peak voltage at Pz")
            
        plt.show()

In [ ]:
correlations_list = []

for participant in participant_numbers:
    participant_epochs = stimulus_epochs['participant == %i' %  participant]
    
    for condition in conditions:
        condition_epochs = participant_epochs['condition == "%s"' %  condition]

        for partner in partners:
            partner_epochs = condition_epochs['partner == "%s"' %  partner]
            
            corr, _ = stats.pearsonr(partner_epochs.metadata['Pz_highest_peak_voltage'], partner_epochs.metadata['participant_confidence'])
            correlations_list.append(corr)
            
            
print('Average correlation between highest peak voltage at Pz and reported confidence: %.4f' % np.mean(correlations_list))            

In [ ]:
correlations_list

In [ ]:
correlations_list = []
correct_voltage_list = []
incorrect_voltage_list = []

for participant in participant_numbers:
    participant_epochs = stimulus_epochs['participant == %i' %  participant]
    correct_epochs = participant_epochs['participant_correct == True']
    correct_voltage_list.append(np.mean(correct_epochs.metadata['Pz_highest_peak_voltage']))
    incorrect_epochs = participant_epochs['participant_correct == False']
    incorrect_voltage_list.append(np.mean(incorrect_epochs.metadata['Pz_highest_peak_voltage']))
    
    for condition in conditions:
        condition_epochs = participant_epochs['condition == "%s"' %  condition]

        for partner in partners:
            partner_epochs = condition_epochs['partner == "%s"' %  partner]
            
            corr, _ = stats.pointbiserialr(partner_epochs.metadata['Pz_highest_peak_voltage'], partner_epochs.metadata['participant_correct'])
            correlations_list.append(corr)
            
            
print('\nAverage point biserial correlation between highest peak voltage at Pz and accuracy: %.4f' % np.mean(correlations_list))  
print('\nAverage highest peak voltage on correct trials: %.4f' % np.mean(correct_voltage_list))
print('\nAverage highest peak voltage on inorrect trials: %.4f' % np.mean(incorrect_voltage_list))

In [ ]:
correct_epochs = stimulus_epochs['participant_correct == True']
incorrect_epochs = stimulus_epochs['participant_correct == False']
x1 = correct_epochs.metadata['Pz_highest_peak_voltage']
x2 = incorrect_epochs.metadata['Pz_highest_peak_voltage']

plt.title("highest peak voltage", fontsize=18)
plt.xlabel("highest peak voltage at Pz 250-1000ms post stimulus")
plt.ylabel("count")
bins = np.arange(stimulus_epochs.metadata['Pz_highest_peak_voltage'].min(), stimulus_epochs.metadata['Pz_highest_peak_voltage'].max()+1, 1)
plt.xlim((-10, 40))
plt.hist(x1, bins=bins, alpha=0.5, color='green', label='correct trials')
plt.axvline(x1.mean(), color='green', linestyle='dashed', linewidth=2)
plt.text(s='%.2f' % x1.mean(), y=620, x=x1.mean()+2, color='green')
plt.hist(x2, bins=bins, alpha=0.5, color='red', label='incorrect trials')
plt.axvline(x2.mean(), color='red', linestyle='dashed', linewidth=2)
plt.text(s='%.2f' % x2.mean(), y=620, x=x2.mean()-6, color='red')
plt.legend()
plt.show() 

## 

## Calculate first peak amplitude and latency post stimulus at Pz

### i.e. instead of looking for the max voltage we look for peaks and take the first one

In [ ]:
tmax = 1000
tmin = 250

indexmax = round(int((300+tmax)/4)) # index 0 is at -300ms; to convert index to ms, conider that this is at 250Hz, i.e. 4ms increase means 1 index increase
indexmin= round(int((300+tmin)/4))
i = 0

for epoch in stimulus_epochs_Pz:
    epoch = epoch.flatten()
    epoch = epoch[indexmin:indexmax] 
    indexes, properties = signal.find_peaks(epoch, height=[-100, 100])
    first_peak = round(properties['peak_heights'][0] * 1e6, 5)
    latency = round(indexes[0]*4+tmin, 5)
    stimulus_epochs.metadata.loc[i, 'Pz_first_peak_voltage'] = first_peak
    stimulus_epochs.metadata.loc[i, 'Pz_first_peak_latency'] = latency
    channel = "Pz"
    print('Trial {}: first peak of {} µV at {} ms in channel {}'
          .format(i, first_peak, latency, channel))
    i +=1



In [ ]:
plt.title("first peak voltage", fontsize=18)
plt.xlabel("first peak voltage at Pz 250-1000ms post stimulus")
plt.ylabel("count")
bins = np.arange(stimulus_epochs.metadata['Pz_first_peak_voltage'].min(), stimulus_epochs.metadata['Pz_first_peak_voltage'].max()+1, 1)
plt.xlim((-10, 40))
plt.hist(stimulus_epochs.metadata['Pz_first_peak_voltage'], bins=bins)
plt.show() 

In [ ]:
print("Mean first peak voltage: %.2f \n\nMedian first peak voltage: %.2f \n\nMax first peak voltage: %.2f \n\nMin first peak voltage: %.2f \n" 
      % (np.mean(stimulus_epochs.metadata['Pz_first_peak_voltage']), np.median(stimulus_epochs.metadata['Pz_first_peak_voltage']),
        np.max(stimulus_epochs.metadata['Pz_first_peak_voltage']), np.min(stimulus_epochs.metadata['Pz_first_peak_voltage'])))

In [ ]:
plt.title("latency", fontsize=18)
plt.xlabel("latency of first peak at Pz 250-1000ms post stimulus")
plt.ylabel("count")
bins = np.arange(stimulus_epochs.metadata['Pz_first_peak_latency'].min(), stimulus_epochs.metadata['Pz_first_peak_latency'].max()+4, 4)
plt.hist(stimulus_epochs.metadata['Pz_first_peak_latency'], bins=bins)
plt.show() 

In [ ]:
print("Mean first peak latency: %.2f ms post stimulus\n \nMedian first peak latency: %.2f ms post stimulus \n \nMax first peak latency: %.2f ms post stimulus \n \nMin first peak latency: %.2f ms post stimulus \n" % 
      (np.mean(stimulus_epochs.metadata['Pz_first_peak_latency']), np.median(stimulus_epochs.metadata['Pz_first_peak_latency']),
      np.max(stimulus_epochs.metadata['Pz_first_peak_latency']), np.min(stimulus_epochs.metadata['Pz_first_peak_latency'])))

In [ ]:
print("\n----------------------- all participants -----------------------\n\n")
for condition in conditions:
    condition_epochs = stimulus_epochs['condition == "%s"' %  condition]
    if condition == "ns":
        print("observational condition")
    else:
        print("strategic condition")
    
    for partner in partners:
        partner_epochs = condition_epochs['partner == "%s"' %  partner]
        x=partner_epochs.metadata['confidence_z_by_participant']
        y=partner_epochs.metadata['Pz_first_peak_voltage']
        slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
        line = slope*x+intercept
        #plt.plot(x, line, label='{} y={:.2f}x+{:.2f}'.format(partner,slope,intercept))
        plt.plot(x, line, label='%s' % partner)
        plt.xlim(-4, 4)
        plt.scatter(x,y, color="k", s=2, alpha=0.2)
        plt.legend(fontsize=13)
        plt.xlabel("reported confidence z-scored by participant")
        plt.ylabel("first peak voltage at Pz")
    
    plt.show()

In [ ]:
for participant in participant_numbers:
    participant_epochs = stimulus_epochs['participant == %i' %  participant]
    print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % participant)
    print('\033[0m')
    
    for condition in conditions:
        condition_epochs = participant_epochs['condition == "%s"' %  condition]
        if condition == "ns":
            print("observational condition")
        else:
            print("strategic condition")

        for partner in partners:
            partner_epochs = condition_epochs['partner == "%s"' %  partner]
            
            x=partner_epochs.metadata['participant_confidence']
            y=partner_epochs.metadata['Pz_first_peak_voltage']
            
            slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
            line = slope*x+intercept
            
            #plt.plot(x, line, label='y={:.2f}x+{:.2f}'.format(slope,intercept))
            plt.plot(x, line, label='%s' % partner)
            plt.xlim(50, 100)
            plt.scatter(x,y, color="k", s=2, alpha=0.5)
            plt.legend(fontsize=13)
            plt.xlabel("reported confidence")
            plt.ylabel("first peak voltage at Pz")
            
        plt.show()

In [ ]:
correlations_list = []

for participant in participant_numbers:
    participant_epochs = stimulus_epochs['participant == %i' %  participant]
    
    for condition in conditions:
        condition_epochs = participant_epochs['condition == "%s"' %  condition]

        for partner in partners:
            partner_epochs = condition_epochs['partner == "%s"' %  partner]
            
            corr, _ = stats.pearsonr(partner_epochs.metadata['Pz_first_peak_voltage'], partner_epochs.metadata['participant_confidence'])
            correlations_list.append(corr)
            
            
print('Average correlation between first peak voltage at Pz and reported confidence: %.4f' % np.mean(correlations_list))            

In [ ]:
correlations_list

In [ ]:
correlations_list = []
correct_voltage_list = []
incorrect_voltage_list = []

for participant in participant_numbers:
    participant_epochs = stimulus_epochs['participant == %i' %  participant]
    correct_epochs = participant_epochs['participant_correct == True']
    correct_voltage_list.append(np.mean(correct_epochs.metadata['Pz_first_peak_voltage']))
    incorrect_epochs = participant_epochs['participant_correct == False']
    incorrect_voltage_list.append(np.mean(incorrect_epochs.metadata['Pz_first_peak_voltage']))
    
    for condition in conditions:
        condition_epochs = participant_epochs['condition == "%s"' %  condition]

        for partner in partners:
            partner_epochs = condition_epochs['partner == "%s"' %  partner]
            
            corr, _ = stats.pointbiserialr(partner_epochs.metadata['Pz_first_peak_voltage'], partner_epochs.metadata['participant_correct'])
            correlations_list.append(corr)
            
            
print('\nAverage point biserial correlation between first peak voltage at Pz and accuracy: %.4f' % np.mean(correlations_list))  
print('\nAverage first peak voltage on correct trials: %.4f' % np.mean(correct_voltage_list))
print('\nAverage first peak voltage on inorrect trials: %.4f' % np.mean(incorrect_voltage_list))

In [ ]:
correct_epochs = stimulus_epochs['participant_correct == True']
incorrect_epochs = stimulus_epochs['participant_correct == False']
x1 = correct_epochs.metadata['Pz_first_peak_voltage']
x2 = incorrect_epochs.metadata['Pz_first_peak_voltage']

plt.title("first peak voltage", fontsize=18)
plt.xlabel("first peak voltage at Pz 250-1000ms post stimulus")
plt.ylabel("count")
bins = np.arange(stimulus_epochs.metadata['Pz_first_peak_voltage'].min(), stimulus_epochs.metadata['Pz_first_peak_voltage'].max()+1, 1)
plt.xlim((-10, 40))
plt.hist(x1, bins=bins, alpha=0.5, color='green', label='correct trials')
plt.axvline(x1.mean(), color='green', linestyle='dashed', linewidth=2)
plt.text(s='%.2f' % x1.mean(), y=520, x=x1.mean()+2, color='green')
plt.hist(x2, bins=bins, alpha=0.5, color='red', label='incorrect trials')
plt.axvline(x2.mean(), color='red', linestyle='dashed', linewidth=2)
plt.text(s='%.2f' % x2.mean(), y=520, x=x2.mean()-6, color='red')
plt.legend()
plt.show() 

## 

## Calculate average voltage post stimulus at Pz

In [ ]:
tmax = 1000
tmin = 250

indexmax = round(int((300+tmax)/4)) # index 0 is at -300ms; to convert index to ms, conider that this is at 250Hz, i.e. 4ms increase means 1 index increase
indexmin= round(int((300+tmin)/4))
i = 0

for epoch in stimulus_epochs_Pz:
    epoch = epoch.flatten()
    epoch = epoch[indexmin:indexmax] 
    average = round(np.mean(epoch) * 1e6, 5)
    stimulus_epochs.metadata.loc[i, 'Pz_average_voltage_250_1000'] = average
    channel = "Pz"
    print('Trial {}: average of {} µV in channel {}'
          .format(i, average, channel))
    i +=1


In [ ]:
plt.title("average voltage", fontsize=18)
plt.xlabel("avergae voltage at Pz 250-1000ms post stimulus")
plt.ylabel("count")
bins = np.arange(stimulus_epochs.metadata['Pz_average_voltage_250_1000'].min(), stimulus_epochs.metadata['Pz_average_voltage_250_1000'].max()+1, 1)
plt.xlim((-10, 40))
plt.hist(stimulus_epochs.metadata['Pz_average_voltage_250_1000'], bins=bins)
plt.show() 

In [ ]:
print("Mean average voltage peak voltage: %.2f \n\nMedian average voltage peak voltage: %.2f \n\nMax average voltage peak voltage: %.2f \n\nMin average voltage peak voltage: %.2f \n" 
      % (np.mean(stimulus_epochs.metadata['Pz_average_voltage_250_1000']), np.median(stimulus_epochs.metadata['Pz_average_voltage_250_1000']),
        np.max(stimulus_epochs.metadata['Pz_average_voltage_250_1000']), np.min(stimulus_epochs.metadata['Pz_average_voltage_250_1000'])))

In [ ]:
print("\n----------------------- all participants -----------------------\n\n")
for condition in conditions:
    condition_epochs = stimulus_epochs['condition == "%s"' %  condition]
    if condition == "ns":
        print("observational condition")
    else:
        print("strategic condition")
    
    for partner in partners:
        partner_epochs = condition_epochs['partner == "%s"' %  partner]
        x=partner_epochs.metadata['confidence_z_by_participant']
        y=partner_epochs.metadata['Pz_average_voltage_250_1000']
        slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
        line = slope*x+intercept
        #plt.plot(x, line, label='{} y={:.2f}x+{:.2f}'.format(partner,slope,intercept))
        plt.plot(x, line, label='%s' % partner)
        plt.xlim(-4, 4)
        plt.scatter(x,y, color="k", s=2, alpha=0.2)
        plt.legend(fontsize=13)
        plt.xlabel("reported confidence z-scored by participant")
        plt.ylabel("average voltage at Pz")
    
    plt.show()

In [ ]:
for participant in participant_numbers:
    participant_epochs = stimulus_epochs['participant == %i' %  participant]
    print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % participant)
    print('\033[0m')
    
    for condition in conditions:
        condition_epochs = participant_epochs['condition == "%s"' %  condition]
        if condition == "ns":
            print("observational condition")
        else:
            print("strategic condition")

        for partner in partners:
            partner_epochs = condition_epochs['partner == "%s"' %  partner]
            
            x=partner_epochs.metadata['participant_confidence']
            y=partner_epochs.metadata['Pz_average_voltage_250_1000']
            
            slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
            line = slope*x+intercept
            
            #plt.plot(x, line, label='y={:.2f}x+{:.2f}'.format(slope,intercept))
            plt.plot(x, line, label='%s' % partner)
            plt.xlim(50, 100)
            plt.scatter(x,y, color="k", s=2, alpha=0.5)
            plt.legend(fontsize=13)
            plt.xlabel("reported confidence")
            plt.ylabel("average voltage at Pz")
            
        plt.show()

In [ ]:
correlations_list = []

for participant in participant_numbers:
    participant_epochs = stimulus_epochs['participant == %i' %  participant]
    
    for condition in conditions:
        condition_epochs = participant_epochs['condition == "%s"' %  condition]

        for partner in partners:
            partner_epochs = condition_epochs['partner == "%s"' %  partner]
            
            corr, _ = stats.pearsonr(partner_epochs.metadata['Pz_average_voltage_250_1000'], partner_epochs.metadata['participant_confidence'])
            correlations_list.append(corr)
            
            
print('Average correlation between average voltage at Pz and reported confidence: %.4f' % np.mean(correlations_list))            

In [ ]:
correlations_list = []
correct_voltage_list = []
incorrect_voltage_list = []

for participant in participant_numbers:
    participant_epochs = stimulus_epochs['participant == %i' %  participant]
    correct_epochs = participant_epochs['participant_correct == True']
    correct_voltage_list.append(np.mean(correct_epochs.metadata['Pz_average_voltage_250_1000']))
    incorrect_epochs = participant_epochs['participant_correct == False']
    incorrect_voltage_list.append(np.mean(incorrect_epochs.metadata['Pz_average_voltage_250_1000']))
    
    for condition in conditions:
        condition_epochs = participant_epochs['condition == "%s"' %  condition]

        for partner in partners:
            partner_epochs = condition_epochs['partner == "%s"' %  partner]
            
            corr, _ = stats.pointbiserialr(partner_epochs.metadata['Pz_average_voltage_250_1000'], partner_epochs.metadata['participant_correct'])
            correlations_list.append(corr)
            
            
print('\nAverage point biserial correlation between average voltage at Pz and accuracy: %.4f' % np.mean(correlations_list))  
print('\nAverage average voltage on correct trials: %.4f' % np.mean(correct_voltage_list))
print('\nAverage average voltage on inorrect trials: %.4f' % np.mean(incorrect_voltage_list))

In [ ]:
correct_epochs = stimulus_epochs['participant_correct == True']
incorrect_epochs = stimulus_epochs['participant_correct == False']
x1 = correct_epochs.metadata['Pz_average_voltage_250_1000']
x2 = incorrect_epochs.metadata['Pz_average_voltage_250_1000']

plt.title("average voltage", fontsize=18)
plt.xlabel("average voltage at Pz 250-1000ms post stimulus")
plt.ylabel("count")
bins = np.arange(stimulus_epochs.metadata['Pz_average_voltage_250_1000'].min(), stimulus_epochs.metadata['Pz_average_voltage_250_1000'].max()+1, 1)
plt.xlim((-10, 40))
plt.hist(x1, bins=bins, alpha=0.5, color='green', label='correct trials')
plt.axvline(x1.mean(), color='green', linestyle='dashed', linewidth=2)
plt.text(s='%.2f' % x1.mean(), y=700, x=x1.mean()+2, color='green')
plt.hist(x2, bins=bins, alpha=0.5, color='red', label='incorrect trials')
plt.axvline(x2.mean(), color='red', linestyle='dashed', linewidth=2)
plt.text(s='%.2f' % x2.mean(), y=700, x=x2.mean()-6, color='red')
plt.legend()
plt.show() 

## 

## Save csv

In [ ]:
df = stimulus_epochs.metadata
df.to_csv('mergedData/stimulus_epochs_mastoids.csv')

## P3 by decision reaction time

In [ ]:
## exclude reaction times > x? 

In [ ]:
zscore = lambda x: (x - x.mean()) / x.std()
# zscore by participant
stimulus_epochs.metadata['decision_rt_z_by_participant'] = stimulus_epochs.metadata['decision_rt'].groupby(stimulus_epochs.metadata['participant']).transform(zscore)


In [ ]:
slow_epochs = stimulus_epochs['decision_rt_z_by_participant >= %i' % 0]
fast_epochs = stimulus_epochs['decision_rt_z_by_participant < %i' % 0]

slow_evoked = slow_epochs.average()
fast_evoked = fast_epochs.average()

evokeds = dict(fast=fast_evoked, slow=slow_evoked)
mne.viz.plot_compare_evokeds(evokeds, picks=['Pz'], invert_y=False)

In [ ]:
print("\n----------------------- all participants -----------------------\n\n")
for condition in conditions:
    condition_epochs = stimulus_epochs['condition == "%s"' %  condition]
    if condition == "ns":
        print("observational condition")
    else:
        print("strategic condition")
    
    for partner in partners:
        partner_epochs = condition_epochs['partner == "%s"' %  partner]
        x=partner_epochs.metadata['decision_rt_z_by_participant']
        y=partner_epochs.metadata['Pz_highest_peak_voltage']
        slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
        line = slope*x+intercept
        #plt.plot(x, line, label='{} y={:.2f}x+{:.2f}'.format(partner,slope,intercept))
        plt.plot(x, line, label='%s' % partner)
        plt.xlim(-2, 4)
        plt.ylim(-10, 20)
        plt.scatter(x,y, color="k", s=2, alpha=0.2)
        plt.legend(fontsize=13)
        plt.xlabel("decision reaction time z-scored by participant")
        plt.ylabel("highest peak voltage at Pz")
    
    plt.show()

In [ ]:
for participant in participant_numbers:
    participant_epochs = stimulus_epochs['participant == %i' %  participant]
    print('\033[1m' + "\n\n----------------------- Participant %i -----------------------\n" % participant)
    print('\033[0m')
    
    for condition in conditions:
        condition_epochs = participant_epochs['condition == "%s"' %  condition]
        if condition == "ns":
            print("observational condition")
        else:
            print("strategic condition")

        for partner in partners:
            partner_epochs = condition_epochs['partner == "%s"' %  partner]
            
            x=partner_epochs.metadata['decision_rt']
            y=partner_epochs.metadata['Pz_highest_peak_voltage']
            
            slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
            line = slope*x+intercept
            
            #plt.plot(x, line, label='y={:.2f}x+{:.2f}'.format(slope,intercept))
            plt.plot(x, line, label='%s' % partner)
            #plt.xlim(0, 3)
            plt.scatter(x,y, color="k", s=2, alpha=0.5)
            plt.legend(fontsize=13)
            plt.xlabel("decision reaction time")
            plt.ylabel("highest peak voltage at Pz")
            
        plt.show()